<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#-Read-the-file-" data-toc-modified-id="-Read-the-file--1"><span class="toc-item-num">1&nbsp;&nbsp;</span> Read the file </a></span></li><li><span><a href="#Mapping" data-toc-modified-id="Mapping-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Mapping</a></span></li></ul></div>

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import math
import os

<h2> Read the file </h2>

In [24]:
# Read datafile corresponding to territorial emissions
#df = pd.read_csv("https://github.com/OpenGeoScales/ogs-data-exploration/tree/main/data/ghg-emissions/owid/owid-co2-data.csv", sep = ";")
path = os.getcwd()
subfolder = '/../data/'
df = pd.read_csv(path+subfolder+'owid-co2-data.csv', sep = ";")

In [13]:
df.head()

,iso_code,country,year,co2,co2_growth_prct,co2_growth_abs,consumption_co2,trade_co2,trade_co2_share,co2_per_capita,...,ghg_per_capita,methane,methane_per_capita,nitrous_oxide,nitrous_oxide_per_capita,primary_energy_consumption,energy_per_capita,energy_per_gdp,population,gdp
0,AFG,Afghanistan,1949,0.015,NaN,NaN,NaN,NaN,NaN,0.002,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7663783.0,NaN
1,AFG,Afghanistan,1950,0.084,475.000,0.070,NaN,NaN,NaN,0.011,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7752000.0,1.949480e+10
2,AFG,Afghanistan,1951,0.092,8.696,0.007,NaN,NaN,NaN,0.012,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7840000.0,2.006385e+10
3,AFG,Afghanistan,1952,0.092,NaN,NaN,NaN,NaN,NaN,0.012,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7936000.0,2.074235e+10
4,AFG,Afghanistan,1953,0.106,16.000,0.015,NaN,NaN,NaN,0.013,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8040000.0,2.201546e+10


In [25]:
# Set the dataframe structure
df.sort_values(["year","country"],inplace=True)
df.set_index(["year","country"],inplace=True)

# Convert the MtCO2 in MtC (to be in adequation with the values from other dataset of GCP)
df[["co2", "trade_co2", "cement_co2",
    "coal_co2", "flaring_co2",
    "gas_co2", "oil_co2", "other_industry_co2"]] = (1.0/3.664)* df[["co2", "trade_co2", "cement_co2",
                                                                    "coal_co2", "flaring_co2",
                                                                    "gas_co2", "oil_co2", "other_industry_co2"]]

df.tail(3)

iso_code       co2  co2_growth_prct  co2_growth_abs  \
year country                                                        
2019 Yemen         YEM  2.798854            3.115           0.310   
     Zambia        ZMB  1.834061           -3.025          -0.210   
     Zimbabwe      ZWE  2.831332           -8.521          -0.966   

               consumption_co2  trade_co2  trade_co2_share  co2_per_capita  \
year country                                                                 
2019 Yemen                 NaN        NaN              NaN           0.352   
     Zambia                NaN        NaN              NaN           0.376   
     Zimbabwe              NaN        NaN              NaN           0.708   

               consumption_co2_per_capita  share_global_co2  ...  \
year country                                                 ...   
2019 Yemen                            NaN             0.028  ...   
     Zambia                           NaN             0.018  ...   
     Zimbabwe                         NaN             0.028  ...   

               ghg_per_capita  methane  methane_per_capita  nitrous_oxide  \
year country                                                                
2019 Yemen                NaN      NaN                 NaN            NaN   
     Zambia               NaN      NaN                 NaN            NaN   
     Zimbabwe             NaN      NaN                 NaN            NaN   

               nitrous_oxide_per_capita  primary_energy_consumption  \
year country                                                          
2019 Yemen                          NaN                         NaN   
     Zambia                         NaN                         NaN   
     Zimbabwe                       NaN                         NaN   

               energy_per_capita  energy_per_gdp  population  gdp  
year country                                                       
2019 Yemen                   NaN             NaN  29162000.0  NaN  
     Zambia                  NaN             NaN  17861000.0  NaN  
     Zimbabwe                NaN             NaN  14645000.0  NaN  

[3 rows x 53 columns]

In [26]:
# Select the variables to include in the JSON file
df = df[["iso_code","co2", "trade_co2", "cement_co2","coal_co2", "flaring_co2",
                    "gas_co2", "oil_co2", "other_industry_co2"]]
df

iso_code          co2  trade_co2  cement_co2  \
year country                                                                   
1750 EU-28                           NaN     2.552129        NaN         NaN   
     Europe                          NaN     2.552129        NaN         NaN   
     Europe (excl. EU-27)            NaN     2.552129        NaN         NaN   
     United Kingdom                  GBR     2.552129        NaN         NaN   
     World                      OWID_WRL     2.552129        NaN         NaN   
...                                  ...          ...        ...         ...   
2019 Wallis and Futuna Islands       WLF     0.007915        NaN         NaN   
     World                      OWID_WRL  9945.793668        NaN  426.790666   
     Yemen                           YEM     2.798854        NaN    0.228439   
     Zambia                          ZMB     1.834061        NaN    0.285480   
     Zimbabwe                        ZWE     2.831332        NaN    0.158570   

                                   coal_co2  flaring_co2      gas_co2  \
year country                                                            
1750 EU-28                         2.552129          NaN          NaN   
     Europe                        2.552129          NaN          NaN   
     Europe (excl. EU-27)          2.552129          NaN          NaN   
     United Kingdom                2.552129          NaN          NaN   
     World                         2.552129          NaN          NaN   
...                                     ...          ...          ...   
2019 Wallis and Futuna Islands          NaN          NaN          NaN   
     World                      3919.805404   117.220524  2078.524563   
     Yemen                         0.091976          NaN     0.340884   
     Zambia                        0.512555          NaN          NaN   
     Zimbabwe                      1.720524          NaN          NaN   

                                    oil_co2  other_industry_co2  
year country                                                     
1750 EU-28                              NaN                 NaN  
     Europe                             NaN                 NaN  
     Europe (excl. EU-27)               NaN                 NaN  
     United Kingdom                     NaN                 NaN  
     World                              NaN                 NaN  
...                                     ...                 ...  
2019 Wallis and Futuna Islands     0.007915                 NaN  
     World                      3372.033024           31.419487  
     Yemen                         2.137555                 NaN  
     Zambia                        1.036299                 NaN  
     Zimbabwe                      0.952238                 NaN  

[23708 rows x 9 columns]

In [27]:
df["iso_code"].unique()

array([nan, 'GBR', 'OWID_WRL', 'CAN', 'DEU', 'POL', 'USA', 'BEL', 'FRA',
       'AUT', 'NOR', 'ARM', 'AZE', 'BLR', 'EST', 'GEO', 'HUN', 'KAZ',
       'KGZ', 'LVA', 'LTU', 'MDA', 'RUS', 'ESP', 'TJK', 'TKM', 'UKR',
       'UZB', 'SWE', 'DNK', 'NLD', 'IRL', 'IND', 'ROU', 'CHE', 'AUS',
       'CZE', 'FIN', 'ITA', 'SVK', 'TUR', 'GRC', 'JPN', 'PRT', 'NZL',
       'BGR', 'PER', 'ZAF', 'BIH', 'HRV', 'MNE', 'MKD', 'SRB', 'SVN',
       'ARG', 'IDN', 'MYS', 'MEX', 'VNM', 'CHL', 'TWN', 'CHN', 'BRA',
       'ZWE', 'VEN', 'PRK', 'KOR', 'IRN', 'PHL', 'TTO', 'EGY', 'NGA',
       'DZA', 'TUN', 'ECU', 'COD', 'PRI', 'COL', 'ABW', 'BES', 'SXM',
       'IRQ', 'MOZ', 'BRB', 'BOL', 'MAR', 'MMR', 'ISR', 'LBN', 'SYR',
       'THA', 'URY', 'ALB', 'BHR', 'BRN', 'MDG', 'ISL', 'SAU', 'HKG',
       'ERI', 'CUB', 'ETH', 'GTM', 'NIC', 'LUX', 'BGD', 'KWT', 'PAK',
       'DOM', 'PAN', 'AFG', 'QAT', 'AGO', 'BHS', 'BLZ', 'BMU', 'BDI',
       'CMR', 'CPV', 'CRI', 'CYP', 'DJI', 'SLV', 'GNQ', 'SWZ', 'FRO',
       'FJI', 'GM

In [28]:
# Select only countries (not continents or other entities)
# NB: Kosovo had no official iso_code, and we kept 'OWID_KOS'
df = df.loc[(df["iso_code"].notnull()) & (df["iso_code"] != "OWID_WRL")]

df

iso_code        co2  trade_co2  cement_co2  \
year country                                                                
1750 United Kingdom                 GBR   2.552129        NaN         NaN   
1751 United Kingdom                 GBR   2.552129        NaN         NaN   
1752 United Kingdom                 GBR   2.552948        NaN         NaN   
1753 United Kingdom                 GBR   2.552948        NaN         NaN   
1754 United Kingdom                 GBR   2.554039        NaN         NaN   
...                                 ...        ...        ...         ...   
2019 Vietnam                        VNM  67.606168        NaN    8.775109   
     Wallis and Futuna Islands      WLF   0.007915        NaN         NaN   
     Yemen                          YEM   2.798854        NaN    0.228439   
     Zambia                         ZMB   1.834061        NaN    0.285480   
     Zimbabwe                       ZWE   2.831332        NaN    0.158570   

                                 coal_co2  flaring_co2   gas_co2    oil_co2  \
year country                                                                  
1750 United Kingdom              2.552129          NaN       NaN        NaN   
1751 United Kingdom              2.552129          NaN       NaN        NaN   
1752 United Kingdom              2.552948          NaN       NaN        NaN   
1753 United Kingdom              2.552948          NaN       NaN        NaN   
1754 United Kingdom              2.554039          NaN       NaN        NaN   
...                                   ...          ...       ...        ...   
2019 Vietnam                    36.907751          NaN  5.049127  16.874181   
     Wallis and Futuna Islands        NaN          NaN       NaN   0.007915   
     Yemen                       0.091976          NaN  0.340884   2.137555   
     Zambia                      0.512555          NaN       NaN   1.036299   
     Zimbabwe                    1.720524          NaN       NaN   0.952238   

                                other_industry_co2  
year country                                        
1750 United Kingdom                            NaN  
1751 United Kingdom                            NaN  
1752 United Kingdom                            NaN  
1753 United Kingdom                            NaN  
1754 United Kingdom                            NaN  
...                                            ...  
2019 Vietnam                                   NaN  
     Wallis and Futuna Islands                 NaN  
     Yemen                                     NaN  
     Zambia                                    NaN  
     Zimbabwe                                  NaN  

[20660 rows x 9 columns]

<h2>Mapping</h2>

In [38]:
# Option to select only a subset of countries/year (to check structure)
#list_country=["Afghanistan","Puerto Rico"]
#list_years=[2018,2019]

# Option to generate the full mapped dataset
list_country = df.index.get_level_values(1).unique()

# Definition of the sub-sector list
list_subsectors=["co2", "trade_co2", "cement_co2","coal_co2", "flaring_co2",
                    "gas_co2", "oil_co2", "other_industry_co2"]

# dictionary that will be mapped to .json 
dict_final={}

# key for geo-component/emission (this is a running index without actual "meaning")
index_geo=0

# Loop over geo-component (in this case, countries)
for country in list_country:
    
    # Option to generate the full mapped dataset
    # Define list of years for the country (as temporal coverage varies from one country to another)
    list_years = df.loc[df.index.get_level_values(1) == country].index.get_level_values(0).unique()
    
    # Identify the first year where a value is available for a country
    yr_ini = list_years[0]
    
    # Dictionnary associated to geo-localised unit (country/town/grid)
    dict_country={}

    # create geo_component dictionary
    dict_geo={}
    dict_geo["scale_name"]="country"
    dict_geo["geoComponent_name"]=country
    dict_geo["iso_code"]={"alpha2":None,"alpha3":df.loc[(yr_ini,country),"iso_code"]}
    dict_geo["properties"]={"area":None}
    
    # Create emission list
    # The emission list has to be updated by looping over all possible
    # years, gas, sectors and sub-sectors.
    em_list=[]
    
    
    
    for yr in list_years:
        for subsec in list_subsectors:
            value = df.loc[(yr,country),subsec]
            
            # Only create entry if value is not NaN
            if not math.isnan(value):
                dict_em_yr={}
                dict_em_yr["date"]=yr
                dict_em_yr["sector_name"]="Territorial Emissions"
                dict_em_yr["subsector_name"]=subsec
                dict_em_yr["gas_name"]="CO2"
                dict_em_yr["value"]=value
                dict_em_yr["unit"]="MtC"                
                dict_em_yr["data_source_name"]="OWID"
                em_list.append(dict_em_yr)

    # Add the two dictionaries inside main dictionnary
    dict_country["geoComponent"]=dict_geo
    dict_country["emissions"]=em_list
    dict_final[index_geo]=dict_country
    index_geo+=1


# Optional printing to check structure (comment when running full database!!)
#print(dict_final)  

# Export as a .json
with open("sample.json", "w") as outfile: 
    json.dump(dict_final, outfile)